In [0]:
# Include all the necessary packages, 
import random
import pandas as pd # pandas for data manipulation and analysis
import numpy as np # numpy to handle multi-dimensional arrays and matricies
import matplotlib # matplotlib for pixel plot
import matplotlib.pyplot as plt

In [0]:
from sklearn.model_selection import train_test_split

In [3]:
#Include all the necessary keras libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


# Hyperparameter optimalization

In [4]:
!pip3 install hyperas
!pip3 install hyperopt

In [0]:
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
import numpy as np

In [0]:
def data():
  df = pd.read_csv('fer2013.csv')
  # Convert the pixel column values into a Python list object
  pixels = df['pixels'].tolist()
  emotions = df['emotion'].tolist()

  # The fer2013 database contains 48x48 face images so we create two variables 
  # to store the width and the height of the image
  width, height = 48, 48

  # Convert each pixel set (pixel array) to a 48x48 image and
  # create a list called faces to store each face image
  faces = []
  for pixel_sequence in pixels:
      # Use Python's list comprehension because it's quicker than a single for cycle
      face = [int(pixel) for pixel in pixel_sequence.split(' ')]
      # Reshape face array to matches the 48x48 face images
      face = np.asarray(face).reshape(width, height)
      # Add the converted face image to the faces list
      faces.append(face.astype('uint8'))

  # Convert the list to a numpy array
  faces = np.asarray(faces)
  emotions = np.asarray(emotions)

  # Expanding the dimension of channel for each image
  faces_exp = np.expand_dims(faces, -1)
  # Converting the labels to catergorical matrix
  emotions_categori = pd.get_dummies(df['emotion']).as_matrix() 

  # Create a dictionary for identify the emotion
  emotion_dict = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}

  # Split the train data into a train and validation group (validation = 20% of the train data)
  x_train, x_val, y_train, y_val = train_test_split(faces_exp, emotions_categori, test_size=0.2, random_state=30)
  # Split the train data into a train and test group (test = 10% of the original train data)
  x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.125, random_state=30)

  return x_train, y_train, x_test, y_test

In [0]:
def create_model(x_train,y_train,x_test,y_test):
    from keras.layers import Layer
    from keras import backend as K
    from keras.layers import LeakyReLU

    num_features = 64 
    num_labels = 7 # Seven different emotions
    batch_size = 64 # One batch contains 64 images
    epochs = 1
    width, height = 48, 48  # Image size
 
    n_layer = {{choice([16, 32, 64])}}
    dropout_n = {{uniform(0, 0.5)}}
    dropout_2 = {{uniform(0, 0.5)}}
    act = {{choice(['relu', 'linear', 'tanh'])}}
    optim = {{choice(['rmsprop', 'adam', 'sgd'])}}
    n_batch = {{choice([64, 128, 256])}}
    print('a modell hiperparaméterei: ', n_layer, dropout_n, dropout_2, act, optim, n_batch)

    model = Sequential()
    model.add(Conv2D(n_layer, kernel_size=(3, 3), activation = act, input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
    model.add(Conv2D(n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(dropout_n))

    model.add(Conv2D(2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(dropout_n))

    model.add(Conv2D(2*2*2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*2*n_layer, kernel_size=(3, 3), activation = act, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(dropout_n))

    # Transform the Inputs to row vector
    model.add(Flatten())
              
    # With dense, we define the fully-connected layers 
    model.add(Dense(2*2*2*n_layer, activation = act))
    model.add(Dropout(dropout_n))
    model.add(Dense(2*2*n_layer, activation = act))
    model.add(Dropout(dropout_n))
    model.add(Dense(2*n_layer, activation = act))
    model.add(Dropout(dropout_n))
    
    model.add(Dense(num_labels, activation='softmax'))

    model.compile(loss = categorical_crossentropy,
                  optimizer = optim,
                  metrics=['accuracy'])

    lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=3, verbose=1)

    early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto')

    checkpointer = ModelCheckpoint('model.h5', monitor='val_acc', verbose=1, save_best_only=True)

    result = model.fit(np.array(x_train), np.array(y_train),
              batch_size = n_batch,
              epochs = epochs,
              verbose = 2,
              validation_data = (np.array(x_val), np.array(y_val)),
              shuffle = True,
              callbacks = [lr_reducer, early_stopper, checkpointer])

    
    # az epoch-ok közül a legnagyobb val_acc elmentése
    best_val_acc = np.amax(result.history['val_acc']) 
    print('legjobb val_acc:', best_val_acc)  
    
    # log kiírása: háló struktúra, és az eredmény
    with open('emotionrecopt.csv', 'a') as csv_file:
      csv_file.write(str(n_layer) + ';')
      csv_file.write(str(dropout_n) + ';')
      csv_file.write(str(act) + ';')
      csv_file.write(str(optim) + ';')
      csv_file.write(str(n_batch) + ';')
      csv_file.write(str(best_val_acc) + '\n')


    # negatív val_acc, mert a hyperopt csomag mindig minimalizál
    return {'loss': -best_val_acc, 'status': STATUS_OK, 'model': model}

In [0]:
import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice,uniform

In [35]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          notebook_name='emotionrecopt',
                                          trials=Trials())

>>> Imports:
#coding=utf-8

try:
    import random
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
except:
    pass

try:
    from keras.losses import categorical_crossentropy
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
except:
    pass

try:
    from keras.models import load_model
except:
    pass

try:
    import keras
except:


/usr/local/lib/python3.6/dist-packages/hyperas/utils.py:149: UserWarning: Inconsistent indentation detected.Found "  " (length: 2) as well as "    " (length: 4)
  indent, len(indent), new_indent, len(new_indent)))
/usr/local/lib/python3.6/dist-packages/hyperas/utils.py:149: UserWarning: Inconsistent indentation detected.Found "    " (length: 4) as well as "  " (length: 2)
  indent, len(indent), new_indent, len(new_indent)))


a modell hiperparaméterei: 
32
0.3671073489296299
0.3462695171578595
linear
sgd
64
  0%|          | 0/5 [00:00<?, ?it/s, best loss: ?]

/content/temp_model.py:166: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  emotions_categori = pd.get_dummies(df['emotion']).as_matrix()


Train on 25120 samples, validate on 7178 samples
Epoch 1/1
 - 18s - loss: 2.8561 - acc: 0.1959 - val_loss: 1.8101 - val_acc: 0.2487


Epoch 00001: val_acc improved from -inf to 0.24868, saving model to model.h5
legjobb val_acc:
0.2486765115631095
a modell hiperparaméterei: 
64
0.25764043248072455
0.21261430843422813
tanh
rmsprop
128
Train on 25120 samples, validate on 7178 samples
Epoch 1/1
 - 22s - loss: 2.0620 - acc: 0.1928 - val_loss: 1.9509 - val_acc: 0.1712


Epoch 00001: val_acc improved from -inf to 0.17122, saving model to model.h5
legjobb val_acc:
0.1712176093640152
a modell hiperparaméterei: 
64
0.23634271098298437
0.3637790828042422
tanh
adam
256
Train on 25120 samples, validate on 7178 samples
Epoch 1/1
 - 23s - loss: 2.0426 - acc: 0.1917 - val_loss: 1.8361 - val_acc: 0.1712


Epoch 00001: val_acc improved from -inf to 0.17122, saving model to model.h5
legjobb val_acc:
0.1712176093640152
a modell hiperparaméterei: 
32
0.38230833827442506
0.20633103640535622
tanh
adam
128
Tr